In [23]:
from datetime import datetime
def parse_time(timestamp):
    """
    'YYYY-MM-DD HH:MM:SS AM/PM', 'YYYY-MM-DD HH:MM:SS', 또는 'YYYY-MM-DD HH:MM' 형식을 파싱하는 함수
    """
    timestamp = timestamp.split('.')[0]
    try:
        # 먼저 24시간 형식으로 파싱 시도
        return datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
    except ValueError:
        try:
            # 실패하면 12시간 형식으로 파싱 시도
            return datetime.strptime(timestamp, '%Y-%m-%d %I:%M:%S %p')
        except ValueError:
            # 실패하면 'YYYY-MM-DD HH:MM' 형식으로 파싱 시도
            return datetime.strptime(timestamp, '%Y-%m-%d %H:%M')

In [24]:
import pandas as pd
bus_df = pd.read_csv("13_bus_down2.csv")
bus_df['Parsed_Date'] = bus_df['Parsed_Date'].apply(parse_time)

In [17]:
len(bus_df)

162582

In [18]:
import pandas as pd
import matplotlib.pyplot as plt

# CSV 파일 읽기 (파일 경로 수정 필요)
df = pd.read_csv('13_bus_down2.csv')
#bus_df['Parsed_Date'] = bus_df['Parsed_Date'].apply(parse_time)

# 새로운 시퀀스 번호를 생성 (MASK_SELECTED == 1일 때마다 시퀀스 ID 증가)
df['sequence_id'] = (df['MASK_SELECTED'] == 29).cumsum() - 1

# 함수 정의: 각 그룹에서 아웃라이어를 찾고 플래그 지정
def flag_outliers(group):
    Q1 = group['travel_time'].quantile(0.2)
    Q3 = group['travel_time'].quantile(0.8)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    group['outlier'] = (group['travel_time'] < lower_bound) | (group['travel_time'] > upper_bound)
    return group

# MASK_SELECTED 기준으로 그룹화하여 아웃라이어 플래그 지정
df = df.groupby('MASK_SELECTED', group_keys=False).apply(flag_outliers)

# 200초 이상인 아웃라이어를 포함한 시퀀스 제거
sequences_with_outliers = df[df['outlier'] & (df['travel_time'] > 200)]['sequence_id'].unique()
filtered_df = df[~df['sequence_id'].isin(sequences_with_outliers)]

# 560초 이상 값이 포함된 시퀀스 제거
sequences_with_high_travel_time = df[df['travel_time'] > 420]['sequence_id'].unique()
filtered_df = filtered_df[~filtered_df['sequence_id'].isin(sequences_with_high_travel_time)]

# 조건을 만족하는 시퀀스 개수 출력
filtered_num_sequences = filtered_df['sequence_id'].nunique()
print(f"조건을 만족하는 시퀀스 개수: {filtered_num_sequences}")

# sequence_id 열 제거 (결과 보기 깔끔하게)
filtered_df = filtered_df.drop(columns=['sequence_id', 'outlier'])


조건을 만족하는 시퀀스 개수: 3211


/tmp/ipykernel_1036637/1225565462.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('MASK_SELECTED', group_keys=False).apply(flag_outliers)


In [71]:
filtered_df = filtered_df[(filtered_df['MASK_SELECTED'] >= 29) & (filtered_df['MASK_SELECTED'] <= 43)]

In [72]:
filtered_df.to_csv('test.csv', index = False)

In [19]:
bus_df = filtered_df

In [20]:
import numpy as np

# 새로운 시퀀스 번호를 생성하기 위해 누적 카운트를 사용하여 그룹화
bus_df['sequence_id'] = (bus_df['MASK_SELECTED'] == 29).cumsum() - 1  # 1일 때마다 시퀀스 ID 증가

# 시퀀스 개수 확인
num_sequences = bus_df['sequence_id'].nunique()

print(f"총 시퀀스 개수: {num_sequences}")

# travel_time이 360 이상인 시퀀스를 제거
filtered_bus_df = bus_df.groupby('sequence_id').filter(lambda x: x['travel_time'].max() <= 300)

# 조건을 만족하는 시퀀스 개수 확인
filtered_num_sequences = filtered_bus_df['sequence_id'].nunique()
print(f"조건을 만족하는 시퀀스 개수: {filtered_num_sequences}")

# sequence_id 열 삭제 (결과만 깔끔하게 보기 위해)
filtered_bus_df = filtered_bus_df.drop(columns='sequence_id')

# 결과 출력
print(filtered_bus_df)


총 시퀀스 개수: 3211
조건을 만족하는 시퀀스 개수: 2251
        MASK_SELECTED Information_Occurrence  BUS_ROUTE  STOP_ORD STOP_NAME  \
0                  29    2020-02-01 06:45:59         13         1     천안아산역   
1                  30    2020-02-01 06:48:46         13         2      호수공원   
2                  31    2020-02-01 06:50:27         13         3       아산역   
3                  32    2020-02-01 06:51:20         13         4    한국표준협회   
4                  33    2020-02-01 06:51:49         13         5  장재마을11단지   
...               ...                    ...        ...       ...       ...   
118112             53    2020-08-31 22:02:25         13        25   대림한숲아파트   
118113             54    2020-08-31 22:04:53         13        26    천안톨게이트   
118114             55    2020-08-31 22:06:03         13        27     상명대학교   
118115             56    2020-08-31 22:06:38         13        28   단국대치대병원   
118116             57    2020-08-31 22:07:17         13        29      단대병원   

          STOP

In [21]:
df = filtered_bus_df

In [12]:
# 새로운 시퀀스 번호를 생성 (MASK_SELECTED == 1일 때마다 시퀀스 ID 증가)
df['sequence_id'] = (df['MASK_SELECTED'] == 29).cumsum() - 1

# 함수 정의: 각 그룹에서 아웃라이어를 찾고 플래그 지정
def flag_outliers(group):
    Q1 = group['travel_time'].quantile(0.2)
    Q3 = group['travel_time'].quantile(0.8)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    group['outlier'] = (group['travel_time'] < lower_bound) | (group['travel_time'] > upper_bound)
    return group

# MASK_SELECTED 기준으로 그룹화하여 아웃라이어 플래그 지정
df = df.groupby('MASK_SELECTED', group_keys=False).apply(flag_outliers)

# 200초 이상인 아웃라이어를 포함한 시퀀스 제거
sequences_with_outliers = df[df['outlier'] & (df['travel_time'] > 200)]['sequence_id'].unique()
filtered_df = df[~df['sequence_id'].isin(sequences_with_outliers)]

# 560초 이상 값이 포함된 시퀀스 제거
sequences_with_high_travel_time = df[df['travel_time'] > 420]['sequence_id'].unique()
filtered_df = filtered_df[~filtered_df['sequence_id'].isin(sequences_with_high_travel_time)]

# 조건을 만족하는 시퀀스 개수 출력
filtered_num_sequences = filtered_df['sequence_id'].nunique()
print(f"조건을 만족하는 시퀀스 개수: {filtered_num_sequences}")

# sequence_id 열 제거 (결과 보기 깔끔하게)
filtered_df = filtered_df.drop(columns=['sequence_id', 'outlier'])


조건을 만족하는 시퀀스 개수: 3198


/tmp/ipykernel_1036637/3416309975.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('MASK_SELECTED', group_keys=False).apply(flag_outliers)


In [22]:
filtered_bus_df.to_csv('../test_ll.csv', index = False)

In [76]:
filtered_df = filtered_bus_df[(filtered_bus_df['MASK_SELECTED'] >= 29) & (filtered_bus_df['MASK_SELECTED'] <= 43)]

In [77]:
filtered_df.to_csv('test.csv')

In [87]:
filtered_df['sequence_id'] = (filtered_df['MASK_SELECTED'] == 29).cumsum() - 1
# 400개의 시퀀스만 선택
selected_sequences = (
    filtered_df[filtered_df['sequence_id'].isin(range(300))]
)

# sequence_id 열 삭제 (결과만 깔끔하게 보기 위해)
selected_sequences = selected_sequences.drop(columns='sequence_id')

# 결과 출력
print(selected_sequences)


       MASK_SELECTED        LAT         LNG    STOP_ID          Parsed_Date  \
0                 29  36.793599  127.103951  288002022  2020-02-01 06:45:59   
1                 30  36.793720  127.101300  288002048  2020-02-01 06:48:46   
2                 31  36.791477  127.103512  288002219  2020-02-01 06:50:27   
3                 32  36.791046  127.107846  288002054  2020-02-01 06:51:20   
4                 33  36.786500  127.108700  288002056  2020-02-01 06:51:49   
...              ...        ...         ...        ...                  ...   
10029             53  36.819375  127.159581  285000706  2020-02-12 14:31:26   
10030             54  36.828264  127.167573  285000643  2020-02-12 14:36:50   
10031             55  36.834304  127.173970  285001052  2020-02-12 14:37:57   
10032             56  36.837034  127.174293  285001595  2020-02-12 14:38:18   
10033             57  36.841241  127.174408  285001715  2020-02-12 14:40:42   

       up_down  temperature  Relative_Humidity  is_

In [88]:
selected_sequences.to_csv('test_s.csv', index = False)